# Vizard v2 Comprehensive Test Suite - Minimal Version

**Sections Included:**
1. Setup & Configuration
2. Dataset Loading & Exploration
3. Core Visual Encodings
4. Range Encodings (X2, Y2)

**Datasets:** CARS, STOCKS, MOVIES from altair.datasets

## 1. Setup & Configuration

### Load imports

In [ ]:
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import pandas as pd
import numpy as np
from altair.datasets import data

### Configure theme

In [ ]:
alt.renderers.enable('html')

@alt.theme.register('bioinformatics_theme', enable=True)
def bioinformatics_theme():
    return alt.theme.ThemeConfig({'width': 600, 'height': 400})

### Load Claude Code magic

In [ ]:
%load_ext vizard_magic

In [ ]:
%%time
%cc --model haiku

### Verify Vizard is working

In [ ]:
%%time
%cc HELP

In [ ]:
%%time
%cc RESET

**Expected:** Display Vizard help documentation with all keywords listed

## 2. Dataset Loading & Exploration

### Dataset 1: CARS

In [ ]:
cars_df = pl.DataFrame(data.cars())
print(f'Shape: {cars_df.shape}')
print(f'Columns: {cars_df.columns}')
cars_df.head(5)

### Dataset 2: STOCKS

In [ ]:
stocks_df = pl.DataFrame(data.stocks())
print(f'Shape: {stocks_df.shape}')
print(f'Columns: {stocks_df.columns}')
stocks_df.head(5)

### Dataset 3: MOVIES

In [ ]:
movies_df = pl.DataFrame(data.movies())
print(f'Shape: {movies_df.shape}')
print(f'Columns: {movies_df.columns}')
movies_df.head(5)

## 3. Core Visual Encodings

Test X, Y, COLOR, SIZE, SHAPE, OPACITY, ROW, COLUMN using CARS dataset.

In [ ]:
%%time
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon

**Expected:** Basic scatter plot showing horsepower vs fuel efficiency

In [ ]:
%%time
%cc COLOR Origin

**Expected:** Points colored by car origin (USA, Europe, Japan)

In [ ]:
%%time
%cc SIZE Weight_in_lbs

**Expected:** Point sizes vary by car weight

In [ ]:
%%time
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLOR Origin SHAPE Origin

**Expected:** Different shapes AND colors for each origin

In [ ]:
%%time
%cc OPACITY Acceleration

**Expected:** Point transparency varies by acceleration

In [ ]:
%%time
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon ROW Cylinders

**Expected:** Multiple scatter plots arranged horizontally, one per cylinder count

In [ ]:
%%time
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLUMN Origin

**Expected:** Three scatter plots stacked vertically, one per origin

In [ ]:
%%time
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLOR Origin ROW Cylinders COLUMN Origin

**Expected:** Multiple scatter plots arranged horizontally, one per cylinder count

## 4. Range Encodings (X2, Y2)

Test range encodings for error bars and intervals.

### Prepare aggregated data with ranges

In [ ]:
# Aggregate cars by Origin AND Cylinders for more bars (10-12 bars)
cars_agg = (
    cars_df
    .filter(pl.col('Cylinders').is_in([4, 6, 8]))  # Focus on common cylinder counts
    .group_by(['Origin', 'Cylinders'])
    .agg([
        pl.col('Miles_per_Gallon').mean().alias('avg_mpg'),
        pl.col('Miles_per_Gallon').min().alias('min_mpg'),
        pl.col('Miles_per_Gallon').max().alias('max_mpg')
    ])
    .with_columns([
        (pl.col('Origin') + ' ' + pl.col('Cylinders').cast(str) + 'cyl').alias('category')
    ])
    .sort(['Origin', 'Cylinders'])
)
print('Aggregated data with ranges by Origin + Cylinders:')
cars_agg

In [ ]:
%%time
%cc RESET
%cc DATA cars_agg PLOT bar X category Y min_mpg Y2 max_mpg TITLE MPG Ranges by Origin and Cylinders

**Expected:** 9-10 bars showing MPG range (min to max) for each Origin+Cylinder combination

In [ ]:
# Create error bar visualization using natural language
%cc Create a chart with points at avg_mpg and error bars from min_mpg to max_mpg

**Expected:** Points with vertical error bars showing full MPG range

In [ ]:
%%time
%cc COLOR Origin

**Expected:** Colored ranges, one color per origin

## 5. Advanced Encodings (SERIES, TEXT)

Test SERIES for multi-series line charts and TEXT for labels.

In [ ]:
%%time
%cc RESET
%cc DATA stocks PLOT line X date Y price SERIES symbol COLOR symbol

**Expected:** Four separate lines (MSFT, AAPL, GOOG, IBM), SERIES groups without adding visual encoding

In [ ]:
# Aggregate cars by origin for simpler visualization
cars_by_origin = (
    cars_df.group_by('Origin')
    .agg(pl.col('Miles_per_Gallon').mean().alias('avg_mpg'))
)
%cc DATA cars_by_origin PLOT bar X Origin Y avg_mpg TEXT avg_mpg

**Expected:** Bar chart with MPG values displayed on top of each bar

### Test 5.3: Scatter with TEXT for high-rated movies

In [ ]:
# Create top movies dataset for TEXT encoding test
top_movies = movies_df.filter(pl.col('IMDB_Rating') > 8.0).head(10)
print(f"Created top_movies: {top_movies.shape[0]} movies with IMDB_Rating > 8.0")
top_movies

In [ ]:
%%time
%cc DATA top_movies PLOT scatter X IMDB_Rating Y IMDB_Votes TEXT Title TITLE Top Rated Movies

In [ ]:
%%time
%cc DATA top_movies PLOT scatter X IMDB_Rating Y IMDB_Votes TEXT Title TITLE Top Rated Movies

**Expected:** Scatter plot with movie titles labeling each point

In [ ]:
%%time
%cc DATA stocks PLOT line X date Y price SERIES symbol COLOR symbol TITLE Stock Prices Over Time

**Expected:** Multi-line chart with distinct colors per stock symbol

## 6. Layout & Grouping (BAR_LAYOUT)

Test stacked, grouped, and normalized bar layouts.

### Prepare aggregated data by Origin and Cylinders

In [ ]:
# Get counts by Origin and Cylinders
cars_counts = (
    cars_df
    .filter(pl.col('Cylinders').is_in([4, 6, 8]))
    .group_by(['Origin', 'Cylinders'])
    .agg(pl.len().alias('count'))
)
print('Car counts by Origin and Cylinders:')
cars_counts

In [ ]:
%%time
%cc RESET
%cc DATA cars_counts PLOT bar X Origin Y count COLOR Cylinders BAR_LAYOUT stacked

**Expected:** Bars stacked vertically, colored by cylinder count

In [ ]:
%%time
%cc BAR_LAYOUT grouped

**Expected:** Bars arranged side-by-side using xOffset

In [ ]:
%%time
%cc BAR_LAYOUT normalized

**Expected:** All bars reach 100%, showing proportions

In [ ]:
%%time
%cc BAR_LAYOUT grouped WIDTH 800 HEIGHT 500 TITLE Car Distribution by Origin and Cylinders

**Expected:** Grouped bars with larger dimensions and title

## 7. Data Transformations (WINDOW)

Test window transformations for running calculations.

In [ ]:
%%time
%cc RESET
%cc DATA stocks PLOT line X date Y price WINDOW cumsum SERIES symbol COLOR symbol TITLE Cumulative Stock Prices

**Expected:** Lines showing cumulative sum of prices over time, one per symbol

In [ ]:
%%time
%cc DATA top_movies PLOT bar X Title Y IMDB_Rating WINDOW rank TITLE Top Movies Ranked

**Expected:** Bar chart with ranking applied via transform_window

In [ ]:
%%time
%cc WINDOW row_number

**Expected:** Sequential numbering applied to data

In [ ]:
%%time
%cc DATA stocks PLOT line X date Y price WINDOW cumsum SERIES symbol COLOR symbol

**Expected:** Separate cumulative sums for each stock symbol

## 8. Meta Commands

Test state management commands: KEYWORDS, KEYS, RESET, HELP

In [ ]:
%%time
%cc KEYWORDS

**Expected:** Display all current keyword values from state

In [ ]:
%%time
%cc KEYS

**Expected:** Same output as KEYWORDS

In [ ]:
%%time
%cc RESET

**Expected:** State cleared, confirmation message

In [ ]:
%%time
%cc --model haiku KEYWORDS

In [ ]:
%%time
%cc HELP

### Test 8.5: Verify RESET worked

In [ ]:
%%time
%cc KEYS

**Expected:** Only default values shown (ENGINE: altair, WIDTH: 600, etc.)

### Test 9.1: IMPORT true - generate imports

In [ ]:
%%time
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon IMPORT

### Test 9.2: IMPORT false (default) - no imports

In [ ]:
%%time
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon

### Test 9.3: FUNCTION - generate reusable function

In [ ]:
%%time
%cc DATA cars PLOT bar X Origin Y Miles_per_Gallon FUNCTION IMPORT

### Test 9.4: FUNCTION + IMPORT together

In [ ]:
%%time
%cc DATA stocks PLOT line X date Y price SERIES symbol FUNCTION IMPORT

**Expected:** Complete function with imports, ready to use

### Test 10.1: Set custom defaults without generating chart

In [ ]:
%%time
%cc RESET
%cc WIDTH 800 HEIGHT 600 DATA cars

### Test 10.2: Generate chart with persisted state

In [ ]:
%%time
%cc PLOT scatter X Horsepower Y Miles_per_Gallon

### Test 10.3: Add COLOR - state accumulates

In [ ]:
%%time
%cc COLOR Origin

### Test 10.4: Verify accumulated state

In [ ]:
%%time
%cc KEYS

### Test 10.5: RESET clears all

In [ ]:
%%time
%cc RESET

### Test 10.6: Verify defaults restored

In [ ]:
%%time
%cc KEYS

### Test 10.7: Dynamic keyword - THRESHOLD

In [ ]:
%%time
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon THRESHOLD 100
%cc Color points where Horsepower > THRESHOLD in red, others in gray

**Expected:** Conditional coloring using custom THRESHOLD keyword

### Test 11.1: Keywords only

In [ ]:
%%time
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon

### Test 11.2: Natural language

In [ ]:
%%time
%cc Create a scatter plot from cars dataset showing horsepower vs fuel efficiency

### Test 11.3: Mixed syntax

In [ ]:
%%time
%cc DATA stocks - create a line chart with X date Y price colored by symbol

### Test 11.4: VZ trigger

In [ ]:
%%time
%cc VZ DATA stocks PLOT line X date Y price SERIES symbol COLOR symbol

### Test 11.5: VIZARD trigger

In [ ]:
%%time
%cc VIZARD make a histogram of Horsepower from cars dataset

In [ ]:
%%time
%cc SIZE Weight_in_lbs

In [ ]:
%%time
%cc TITLE Car Performance Analysis

In [ ]:
%%time
%cc SIZE Weight_in_lbs

In [ ]:
%%time
%cc TITLE Car Performance Analysis

**Expected:** Chart progressively enhanced through multiple cells

print('Movies with null IMDB ratings:', movies_df.filter(pl.col('IMDB Rating').is_null()).shape[0])
%%time
%cc RESET
%cc DATA movies PLOT scatter X Production_Budget Y IMDB_Rating TITLE Budget vs Rating

### Test 12.1: Null handling - movies with missing ratings

In [ ]:
# Check null distribution
print('Movies with null IMDB ratings:', movies_df.filter(pl.col('IMDB Rating').is_null()).shape[0])
%cc RESET
%cc DATA movies PLOT scatter X Production_Budget Y IMDB_Rating TITLE Budget vs Rating

### Test 12.2: Many simultaneous encodings

In [ ]:
%%time
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLOR Origin SIZE Weight_in_lbs OPACITY Acceleration

### Test 12.3: Long column names

In [ ]:
%%time
%cc DATA movies PLOT scatter X Production_Budget Y Rotten_Tomatoes_Rating

### Test 12.4: Datetime encoding

In [ ]:
%%time
%cc DATA stocks PLOT line X date Y price SERIES symbol

### Test 12.5: Very wide chart

In [ ]:
%%time
%cc WIDTH 1200 HEIGHT 300

### Test 12.6: Multiple dynamic keywords

In [ ]:
%%time
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon THRESHOLD_LOW 15 THRESHOLD_HIGH 25
%cc Color points: red if MPG > THRESHOLD_HIGH, blue if MPG < THRESHOLD_LOW, gray otherwise

### Test 12.7: Spelling tolerance

In [ ]:
%%time
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLOUR Origin TITEL Car Analysis HIGHT 450

### Test 12.8: ENGINE switching - matplotlib

In [ ]:
%%time
%cc ENGINE matplotlib

### Test 12.9: ENGINE switching - seaborn

In [ ]:
%%time
%cc ENGINE seaborn

### Test 12.10: Complex WINDOW - grouped by symbol

In [ ]:
%%time
%cc ENGINE altair
%cc DATA stocks PLOT line X date Y price WINDOW cumsum SERIES symbol COLOR symbol TITLE Cumulative Price by Symbol

**Expected:** Each stock symbol has its own cumulative sum line

## Test Summary

**Sections tested:**
1. ✅ Setup & Configuration
2. ✅ Dataset Loading (CARS, STOCKS, MOVIES)
3. ✅ Core Visual Encodings (X, Y, COLOR, SIZE, SHAPE, OPACITY, ROW, COLUMN)
4. ✅ Range Encodings (X2, Y2)
5. ✅ Advanced Encodings (SERIES, TEXT)
6. ✅ Layout & Grouping (BAR_LAYOUT: stacked, grouped, normalized)
7. ✅ Data Transformations (WINDOW: cumsum, rank, row_number)
8. ✅ Meta Commands (KEYWORDS, KEYS, RESET, HELP)
9. ✅ Code Generation (FUNCTION, IMPORT)
10. ✅ State Management & Persistence
11. ✅ Syntax Flexibility (keywords, natural language, mixed)
12. ✅ Edge Cases (nulls, many encodings, datetime, ENGINE switching)

**Total test scenarios:** ~60+